In [1]:
from pyspark.sql import SparkSession
import time

# --- CONFIGURACIÓN ---
DB_HOST = "sql_server"
DB_USER = "sa"
DB_PASS = "PasswordFuerte123!" 
DRIVER_PKG = "com.microsoft.sqlserver:mssql-jdbc:11.2.0.jre8"

print("--- INICIANDO SETUP (MODO DDL DIRECTO) ---")

# 1. Iniciar Spark
spark = SparkSession.builder \
    .appName("Setup-Final") \
    .master("local[*]") \
    .config("spark.jars.packages", DRIVER_PKG) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# 2. REGISTRAR DRIVER MANUALMENTE (El paso que faltaba)
try:
    print(">>> Registrando Driver en la JVM...")
    # Accedemos a la clase del driver directamente desde el Gateway de Spark
    driver_instance = spark.sparkContext._gateway.jvm.com.microsoft.sqlserver.jdbc.SQLServerDriver()
    # Lo registramos en el DriverManager
    spark.sparkContext._gateway.jvm.java.sql.DriverManager.registerDriver(driver_instance)
    print("✅ Driver registrado exitosamente.")
except Exception as e:
    print(f"❌ Error registrando driver: {e}")

# 3. Función para ejecutar SQL Puro (CREATE, INSERT, DROP)
def ejecutar_ddl(query, db_name="master", descripcion=""):
    url = f"jdbc:sqlserver://{DB_HOST}:1433;databaseName={db_name};encrypt=true;trustServerCertificate=true;"
    try:
        # Obtenemos el DriverManager de Java
        manager = spark.sparkContext._gateway.jvm.java.sql.DriverManager
        # Creamos la conexión directa
        con = manager.getConnection(url, DB_USER, DB_PASS)
        stmt = con.createStatement()
        
        # Ejecutamos el comando DDL puro
        stmt.execute(query)
        
        # Cerramos
        stmt.close()
        con.close()
        print(f"   ✅ OK: {descripcion}")
        
    except Exception as e:
        error_str = str(e)
        if "already exists" in error_str:
            print(f"   ℹ️ INFO: {descripcion} (Ya existía)")
        elif "Statement did not return a result set" in error_str:
            # A veces .execute() lanza esto aunque funcione, lo tratamos como éxito
            print(f"   ✅ OK: {descripcion}")
        else:
            print(f"   ❌ ERROR en {descripcion}: {error_str.split(':')[0]}")

# --- EJECUCIÓN ---

print("\n>>> 1. Esperando a SQL Server (5s)...")
time.sleep(5)

print("\n>>> 2. Creando Bases de Datos...")
ejecutar_ddl("IF DB_ID('alquiler_habitacion') IS NULL CREATE DATABASE alquiler_habitacion", "master", "Crear DB alquiler_habitacion")
ejecutar_ddl("IF DB_ID('DW_AlquilerHabitacion') IS NULL CREATE DATABASE DW_AlquilerHabitacion", "master", "Crear DB DW_AlquilerHabitacion")

print("\n>>> 3. Creando Tablas OLTP...")
schemas_oltp = [
    ("CLIENTE", "IF OBJECT_ID('dbo.CLIENTE','U') IS NULL CREATE TABLE dbo.CLIENTE (idCliente INT IDENTITY(1,1) PRIMARY KEY, nombreCli NVARCHAR(100), dniCli CHAR(8), telefonoCli NVARCHAR(20), correoCli NVARCHAR(100), direccionCli NVARCHAR(200), FechaRegistro DATETIME DEFAULT(GETDATE()))"),
    ("HABITACION", "IF OBJECT_ID('dbo.HABITACION','U') IS NULL CREATE TABLE dbo.HABITACION (idHabitacion INT IDENTITY(1,1) PRIMARY KEY, numeroHabitacion NVARCHAR(10), tipoHabitacion NVARCHAR(50), precioMensual DECIMAL(10,2), estado NVARCHAR(20), serviciosIncluidos NVARCHAR(200))"),
    ("PROPIETARIO", "IF OBJECT_ID('dbo.PROPIETARIO','U') IS NULL CREATE TABLE dbo.PROPIETARIO (idPropietario INT IDENTITY(1,1) PRIMARY KEY, dniPropietario CHAR(8), nombrePropietario NVARCHAR(100), celularPropietario NVARCHAR(20), correoPropietario NVARCHAR(100))"),
    ("CONTRATO", "IF OBJECT_ID('dbo.CONTRATO','U') IS NULL CREATE TABLE dbo.CONTRATO (idContrato INT IDENTITY(1,1) PRIMARY KEY, fechaInicio DATE, fechaFin DATE, montoTotal DECIMAL(12,2), idCliente INT, idHabitacion INT, idPropietario INT, EstadoContrato NVARCHAR(20) DEFAULT('Activo'))"),
    ("PAGO", "IF OBJECT_ID('dbo.PAGO','U') IS NULL CREATE TABLE dbo.PAGO (idPago INT IDENTITY(1,1) PRIMARY KEY, fechaPago DATE, montoPago DECIMAL(12,2), metodoPago NVARCHAR(30), idContrato INT)")
]
for nombre, query in schemas_oltp:
    ejecutar_ddl(query, "alquiler_habitacion", f"Tabla {nombre}")

print("\n>>> 4. Creando Tablas DW...")
schemas_dw = [
    ("DIM_CLIENTE", "IF OBJECT_ID('dbo.DIM_CLIENTE','U') IS NULL CREATE TABLE DIM_CLIENTE (idClienteDW INT IDENTITY(1,1) PRIMARY KEY, idCliente INT, nombreCli NVARCHAR(100), dniCli CHAR(8), telefonoCli NVARCHAR(20), correoCli NVARCHAR(100), direccionCli NVARCHAR(200))"),
    ("DIM_HABITACION", "IF OBJECT_ID('dbo.DIM_HABITACION','U') IS NULL CREATE TABLE DIM_HABITACION (idHabitacionDW INT IDENTITY(1,1) PRIMARY KEY, idHabitacion INT, numeroHabitacion NVARCHAR(10), tipoHabitacion NVARCHAR(50), precioMensual DECIMAL(10,2), estado NVARCHAR(20))"),
    ("DIM_PROPIETARIO", "IF OBJECT_ID('dbo.DIM_PROPIETARIO','U') IS NULL CREATE TABLE DIM_PROPIETARIO (idPropietarioDW INT IDENTITY (1,1) PRIMARY KEY, idPropietario INT, nombrePropietario NVARCHAR (100), correoPropietario NVARCHAR (100))"),
    ("DIM_TIEMPO", "IF OBJECT_ID('dbo.DIM_TIEMPO','U') IS NULL CREATE TABLE DIM_TIEMPO (idTiempoDW INT IDENTITY (1,1) PRIMARY KEY, fecha DATE, anio INT, mes INT, dia INT)"),
    ("FACT_PAGO", "IF OBJECT_ID('dbo.FACT_PAGO','U') IS NULL CREATE TABLE FACT_PAGO (idPagoDW INT IDENTITY (1,1) PRIMARY KEY, idClienteDW INT, idHabitacionDW INT, idPropietarioDW INT, idTiempoDW INT, montoPago DECIMAL (12,2), metodoPago NVARCHAR (30))")
]
for nombre, query in schemas_dw:
    ejecutar_ddl(query, "DW_AlquilerHabitacion", f"Tabla DW {nombre}")

print("\n✅ ¡Setup Finalizado Exitosamente!")


--- INICIANDO SETUP (MODO DDL DIRECTO) ---
>>> Registrando Driver en la JVM...
✅ Driver registrado exitosamente.

>>> 1. Esperando a SQL Server (5s)...

>>> 2. Creando Bases de Datos...
   ❌ ERROR en Crear DB alquiler_habitacion: An error occurred while calling z
   ❌ ERROR en Crear DB DW_AlquilerHabitacion: An error occurred while calling z

>>> 3. Creando Tablas OLTP...
   ❌ ERROR en Tabla CLIENTE: An error occurred while calling z
   ❌ ERROR en Tabla HABITACION: An error occurred while calling z
   ❌ ERROR en Tabla PROPIETARIO: An error occurred while calling z
   ❌ ERROR en Tabla CONTRATO: An error occurred while calling z
   ❌ ERROR en Tabla PAGO: An error occurred while calling z

>>> 4. Creando Tablas DW...
   ❌ ERROR en Tabla DW DIM_CLIENTE: An error occurred while calling z
   ❌ ERROR en Tabla DW DIM_HABITACION: An error occurred while calling z
   ❌ ERROR en Tabla DW DIM_PROPIETARIO: An error occurred while calling z
   ❌ ERROR en Tabla DW DIM_TIEMPO: An error occurred while 